In [5]:
import os
import s3fs
import pandas as pd


In [6]:
# (Option) Télécharger une copie locale
LOCAL_TAXONOMY_PATH = "taxonomy.txt"

taxonomy = pd.read_csv(
    LOCAL_TAXONOMY_PATH,
    sep="\t",
    header=None,
    names=["id_path", "label_path"],
    dtype=str
)

In [8]:

# Liste d'IDs et de labels par niveau
taxonomy["id_list"] = taxonomy["id_path"].str.split(" > ")
taxonomy["label_list"] = taxonomy["label_path"].str.split(" > ")

# Profondeur du nœud (nombre de niveaux)
taxonomy["depth"] = taxonomy["label_list"].str.len()

# Nom de la catégorie (dernier élément du chemin)
taxonomy["category_name"] = taxonomy["label_list"].str[-1]

# Catégorie de niveau 1 (top-level)
taxonomy["level1"] = taxonomy["label_list"].str[0]

# Chemin du parent (ROOT si c'est un niveau 1)
def get_parent_path(labels):
    if len(labels) == 1:
        return "ROOT"
    return " > ".join(labels[:-1])

taxonomy["parent_label_path"] = taxonomy["label_list"].apply(get_parent_path)

taxonomy.head()


,id_path,label_path,id_list,label_list,depth,category_name,level1,parent_label_path
0,1,animals & pet supplies,[1],[animals & pet supplies],1,animals & pet supplies,animals & pet supplies,ROOT
1,1 > 3237,animals & pet supplies > live animals,"[1, 3237]","[animals & pet supplies, live animals]",2,live animals,animals & pet supplies,animals & pet supplies
2,1 > 2,animals & pet supplies > pet supplies,"[1, 2]","[animals & pet supplies, pet supplies]",2,pet supplies,animals & pet supplies,animals & pet supplies
3,1 > 2 > 3,animals & pet supplies > pet supplies > bird s...,"[1, 2, 3]","[animals & pet supplies, pet supplies, bird su...",3,bird supplies,animals & pet supplies,animals & pet supplies > pet supplies
4,1 > 2 > 3 > 7385,animals & pet supplies > pet supplies > bird s...,"[1, 2, 3, 7385]","[animals & pet supplies, pet supplies, bird su...",4,bird cage accessories,animals & pet supplies,animals & pet supplies > pet supplies > bird s...


In [9]:
# Bloc 4 : statistiques globales

n_nodes = len(taxonomy)
n_level1 = taxonomy["level1"].nunique()
depth_stats = taxonomy["depth"].describe()
nodes_per_depth = taxonomy["depth"].value_counts().sort_index()

print("Nombre total de nœuds dans la taxonomie :", n_nodes)
print("Nombre de catégories de niveau 1 :", n_level1)
print("\nStatistiques descriptives sur la profondeur :")
print(depth_stats)
print("\nNombre de nœuds par profondeur :")
print(nodes_per_depth)


Nombre total de nœuds dans la taxonomie : 5440
Nombre de catégories de niveau 1 : 34

Statistiques descriptives sur la profondeur :
count    5440.000000
mean        4.071875
std         1.013323
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: depth, dtype: float64

Nombre de nœuds par profondeur :
depth
1      34
2     191
3    1330
4    2142
5    1323
6     378
7      42
Name: count, dtype: int64


In [10]:
# Bloc 5 : distribution des catégories de niveau 1

level1_counts = taxonomy["level1"].value_counts()

print("Top 20 des catégories de niveau 1 (par nombre de nœuds descendants) :\n")
print(level1_counts.head(20))

print("\nProportion des 10 plus grosses catégories (en % du total des nœuds) :")
print((level1_counts.head(10) / n_nodes * 100).round(2))


Top 20 des catégories de niveau 1 (par nombre de nœuds descendants) :

level1
home & garden                1016
sporting goods                799
hardware                      514
arts & entertainment          487
electronics                   404
food, beverages & tobacco     345
health & beauty               331
vehicles & parts              230
apparel & accessories         220
business & industrial         214
toys & games                  170
office supplies               145
animals & pet supplies        121
furniture                     121
cameras & optics              104
baby & toddler                 81
mature                         38
software                       35
luggage & bags                 22
media                          17
Name: count, dtype: int64

Proportion des 10 plus grosses catégories (en % du total des nœuds) :
level1
home & garden                18.68
sporting goods               14.69
hardware                      9.45
arts & entertainment          8.9

In [11]:
# Bloc 6 : feuilles, nœuds internes, enfants par parent

# Ensemble des chemins utilisés comme parents
parent_set = set(taxonomy["parent_label_path"].unique())

# Un nœud est feuille s'il n'apparaît jamais comme parent
taxonomy["is_leaf"] = ~taxonomy["label_path"].isin(parent_set)

n_leaves = taxonomy["is_leaf"].sum()
n_internal = n_nodes - n_leaves

print("Nombre de feuilles :", n_leaves)
print("Nombre de nœuds internes :", n_internal)

# Nombre d'enfants distincts par parent
children_per_parent = taxonomy.groupby("parent_label_path")["category_name"].nunique()

print("\nStatistiques sur le nombre d'enfants par parent :")
print(children_per_parent.describe())

print("\nQuelques parents avec beaucoup d'enfants :")
print(children_per_parent.sort_values(ascending=False).head(15))


Nombre de feuilles : 4625
Nombre de nœuds internes : 815

Statistiques sur le nombre d'enfants par parent :
count    816.000000
mean       6.666667
std        7.561067
min        1.000000
25%        3.000000
50%        5.000000
75%        8.000000
max       79.000000
Name: category_name, dtype: float64

Quelques parents avec beaucoup d'enfants :
parent_label_path
hardware > tools                                                                            79
home & garden > kitchen & dining > kitchen tools & utensils                                 77
home & garden > decor                                                                       75
food, beverages & tobacco > food items > fruits & vegetables > fresh & frozen vegetables    58
home & garden > kitchen & dining > kitchen appliances                                       52
animals & pet supplies > pet supplies                                                       46
food, beverages & tobacco > food items > fruits & vegetables > f

In [12]:
# Bloc 7 : sauvegarder un résumé en CSV (optionnel)

summary_depth = nodes_per_depth.reset_index()
summary_depth.columns = ["depth", "n_nodes"]

summary_level1 = level1_counts.reset_index()
summary_level1.columns = ["level1", "n_nodes"]

summary_depth.to_csv("taxonomy_depth_stats.csv", index=False)
summary_level1.to_csv("taxonomy_level1_stats.csv", index=False)
